In [1]:
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('pandas').getOrCreate()

In [24]:
df=pd.read_json('sales_data.json')
df

,transaction_id,product_category,sale_amount,date
0,1,Electronics,150,2023-10-01
1,2,Clothing,80,2023-10-02
2,3,Electronics,200,2023-10-03
3,4,Home & Kitchen,120,2023-10-04
4,5,Clothing,90,2023-10-05
5,6,Electronics,300,2023-10-06
6,7,Home & Kitchen,70,2023-10-07
7,8,Clothing,110,2023-10-08
8,9,Electronics,250,2023-10-09
9,10,Home & Kitchen,180,2023-10-10


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10 non-null     int64         
 1   product_category  10 non-null     object        
 2   sale_amount       10 non-null     int64         
 3   date              10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 452.0+ bytes


In [7]:
df[df['sale_amount']>100]

,transaction_id,product_category,sale_amount,date
0,1,Electronics,150,2023-10-01
2,3,Electronics,200,2023-10-03
3,4,Home & Kitchen,120,2023-10-04
5,6,Electronics,300,2023-10-06
7,8,Clothing,110,2023-10-08
8,9,Electronics,250,2023-10-09
9,10,Home & Kitchen,180,2023-10-10


In [14]:
res=df.groupby('product_category').agg(total=('sale_amount','sum')).sort_values('total',ascending=False)
res

,total
product_category,
Electronics,900
Home & Kitchen,370
Clothing,280


In [15]:
res.to_csv('product.csv')

In [18]:
df_spark=spark.read.option('multiline','true').format('json').load('sales_data.json')

In [19]:
df_spark.show()

+----------+----------------+-----------+--------------+
|      date|product_category|sale_amount|transaction_id|
+----------+----------------+-----------+--------------+
|2023-10-01|     Electronics|        150|             1|
|2023-10-02|        Clothing|         80|             2|
|2023-10-03|     Electronics|        200|             3|
|2023-10-04|  Home & Kitchen|        120|             4|
|2023-10-05|        Clothing|         90|             5|
|2023-10-06|     Electronics|        300|             6|
|2023-10-07|  Home & Kitchen|         70|             7|
|2023-10-08|        Clothing|        110|             8|
|2023-10-09|     Electronics|        250|             9|
|2023-10-10|  Home & Kitchen|        180|            10|
+----------+----------------+-----------+--------------+



In [20]:
df_spark.printSchema()

root
 |-- date: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- sale_amount: long (nullable = true)
 |-- transaction_id: long (nullable = true)



In [22]:
from pyspark.sql.functions import *
df_spark=df_spark.withColumn('date',to_date('date'))

In [23]:
df_spark.printSchema()

root
 |-- date: date (nullable = true)
 |-- product_category: string (nullable = true)
 |-- sale_amount: long (nullable = true)
 |-- transaction_id: long (nullable = true)



In [26]:
df_spark[df_spark.sale_amount>100].show()

+----------+----------------+-----------+--------------+
|      date|product_category|sale_amount|transaction_id|
+----------+----------------+-----------+--------------+
|2023-10-01|     Electronics|        150|             1|
|2023-10-03|     Electronics|        200|             3|
|2023-10-04|  Home & Kitchen|        120|             4|
|2023-10-06|     Electronics|        300|             6|
|2023-10-08|        Clothing|        110|             8|
|2023-10-09|     Electronics|        250|             9|
|2023-10-10|  Home & Kitchen|        180|            10|
+----------+----------------+-----------+--------------+



In [30]:
result=df_spark.groupBy('product_category').agg(sum('sale_amount').alias('Total')).orderBy('Total',ascending=False)
result.show()

+----------------+-----+
|product_category|Total|
+----------------+-----+
|     Electronics|  900|
|  Home & Kitchen|  370|
|        Clothing|  280|
+----------------+-----+



In [31]:
result.write.parquet('./message/df.parquet_1')

Project

In [55]:
import pandas as pd
import json
from io import StringIO

sales_data = """transaction_id,customer_id,product_id,sale_amount,date
1,101,1001,250.50,2024-01-01
2,102,1002,99.99,2024-03-02
3,103,1003,150.00,2024-06-03
4,101,1004,349.99,2024-05-04
5,104,1001,250.50,2024-09-05
6,105,1005,500.00,2024-06-06
7,106,1002,99.99,2024-11-07
8,107,1003,150.00,2024-12-08
9,108,1004,349.99,2024-08-09
10,109,1005,500.00,2024-03-10"""

products_data = """product_id,product_name,category,price
1001,Laptop,Electronics,999.99
1002,Headphones,Electronics,99.99
1003,Smartphone,Electronics,699.99
1004,Tablet,Electronics,349.99
1005,Smartwatch,Electronics,500.00"""

customers_data = """[
    {"customer_id": 101, "name": "Alice Johnson", "email": "alice@example.com", "location": "New York"},
    {"customer_id": 102, "name": "Bob Smith", "email": "bob@example.com", "location": "Los Angeles"},
    {"customer_id": 103, "name": "Charlie Brown", "email": "charlie@example.com", "location": "Chicago"},
    {"customer_id": 104, "name": "David Lee", "email": "david@example.com", "location": "Houston"},
    {"customer_id": 105, "name": "Emma Davis", "email": "emma@example.com", "location": "Phoenix"},
    {"customer_id": 106, "name": "Frank Wilson", "email": "frank@example.com", "location": "Philadelphia"},
    {"customer_id": 107, "name": "Grace Hall", "email": "grace@example.com", "location": "San Antonio"},
    {"customer_id": 108, "name": "Hannah Moore", "email": "hannah@example.com", "location": "San Diego"},
    {"customer_id": 109, "name": "Ian Taylor", "email": "ian@example.com", "location": "Dallas"}
]"""

In [56]:
sales=pd.read_csv(StringIO(sales_data))
sales

,transaction_id,customer_id,product_id,sale_amount,date
0,1,101,1001,250.50,2024-01-01
1,2,102,1002,99.99,2024-03-02
2,3,103,1003,150.00,2024-06-03
3,4,101,1004,349.99,2024-05-04
4,5,104,1001,250.50,2024-09-05
5,6,105,1005,500.00,2024-06-06
6,7,106,1002,99.99,2024-11-07
7,8,107,1003,150.00,2024-12-08
8,9,108,1004,349.99,2024-08-09
9,10,109,1005,500.00,2024-03-10


In [57]:
product=pd.read_csv(StringIO(products_data))
print(product)
customer=pd.read_json(StringIO(customers_data))
customer

   product_id product_name     category   price
0        1001       Laptop  Electronics  999.99
1        1002   Headphones  Electronics   99.99
2        1003   Smartphone  Electronics  699.99
3        1004       Tablet  Electronics  349.99
4        1005   Smartwatch  Electronics  500.00


,customer_id,name,email,location
0,101,Alice Johnson,alice@example.com,New York
1,102,Bob Smith,bob@example.com,Los Angeles
2,103,Charlie Brown,charlie@example.com,Chicago
3,104,David Lee,david@example.com,Houston
4,105,Emma Davis,emma@example.com,Phoenix
5,106,Frank Wilson,frank@example.com,Philadelphia
6,107,Grace Hall,grace@example.com,San Antonio
7,108,Hannah Moore,hannah@example.com,San Diego
8,109,Ian Taylor,ian@example.com,Dallas


In [58]:
product.isnull().sum()

product_id      0
product_name    0
category        0
price           0
dtype: int64

In [59]:
sales.isnull().sum()

transaction_id    0
customer_id       0
product_id        0
sale_amount       0
date              0
dtype: int64

In [60]:
customer.isnull().sum()

customer_id    0
name           0
email          0
location       0
dtype: int64

In [61]:
print(customer.duplicated())
print(product.duplicated())
sales.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
dtype: bool
0    False
1    False
2    False
3    False
4    False
dtype: bool


0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [62]:
customer.drop_duplicates(inplace=True)

In [63]:
print(customer.info())
print(sales.info())
print(product.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  9 non-null      int64 
 1   name         9 non-null      object
 2   email        9 non-null      object
 3   location     9 non-null      object
dtypes: int64(1), object(3)
memory usage: 360.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  10 non-null     int64  
 1   customer_id     10 non-null     int64  
 2   product_id      10 non-null     int64  
 3   sale_amount     10 non-null     float64
 4   date            10 non-null     object 
dtypes: float64(1), int64(3), object(1)
memory usage: 532.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   C

In [64]:
sales['date']=pd.to_datetime(sales['date'])

In [65]:
df=pd.merge(customer,sales,on='customer_id')
df=pd.merge(df,product,on='product_id')
df

,customer_id,name,email,location,transaction_id,product_id,sale_amount,date,product_name,category,price
0,101,Alice Johnson,alice@example.com,New York,1,1001,250.50,2024-01-01,Laptop,Electronics,999.99
1,104,David Lee,david@example.com,Houston,5,1001,250.50,2024-09-05,Laptop,Electronics,999.99
2,101,Alice Johnson,alice@example.com,New York,4,1004,349.99,2024-05-04,Tablet,Electronics,349.99
3,108,Hannah Moore,hannah@example.com,San Diego,9,1004,349.99,2024-08-09,Tablet,Electronics,349.99
4,102,Bob Smith,bob@example.com,Los Angeles,2,1002,99.99,2024-03-02,Headphones,Electronics,99.99
5,106,Frank Wilson,frank@example.com,Philadelphia,7,1002,99.99,2024-11-07,Headphones,Electronics,99.99
6,103,Charlie Brown,charlie@example.com,Chicago,3,1003,150.00,2024-06-03,Smartphone,Electronics,699.99
7,107,Grace Hall,grace@example.com,San Antonio,8,1003,150.00,2024-12-08,Smartphone,Electronics,699.99
8,105,Emma Davis,emma@example.com,Phoenix,6,1005,500.00,2024-06-06,Smartwatch,Electronics,500.00
9,109,Ian Taylor,ian@example.com,Dallas,10,1005,500.00,2024-03-10,Smartwatch,Electronics,500.00


In [66]:
df['month']=df['date'].dt.month

In [67]:
df

,customer_id,name,email,location,transaction_id,product_id,sale_amount,date,product_name,category,price,month
0,101,Alice Johnson,alice@example.com,New York,1,1001,250.50,2024-01-01,Laptop,Electronics,999.99,1
1,104,David Lee,david@example.com,Houston,5,1001,250.50,2024-09-05,Laptop,Electronics,999.99,9
2,101,Alice Johnson,alice@example.com,New York,4,1004,349.99,2024-05-04,Tablet,Electronics,349.99,5
3,108,Hannah Moore,hannah@example.com,San Diego,9,1004,349.99,2024-08-09,Tablet,Electronics,349.99,8
4,102,Bob Smith,bob@example.com,Los Angeles,2,1002,99.99,2024-03-02,Headphones,Electronics,99.99,3
5,106,Frank Wilson,frank@example.com,Philadelphia,7,1002,99.99,2024-11-07,Headphones,Electronics,99.99,11
6,103,Charlie Brown,charlie@example.com,Chicago,3,1003,150.00,2024-06-03,Smartphone,Electronics,699.99,6
7,107,Grace Hall,grace@example.com,San Antonio,8,1003,150.00,2024-12-08,Smartphone,Electronics,699.99,12
8,105,Emma Davis,emma@example.com,Phoenix,6,1005,500.00,2024-06-06,Smartwatch,Electronics,500.00,6
9,109,Ian Taylor,ian@example.com,Dallas,10,1005,500.00,2024-03-10,Smartwatch,Electronics,500.00,3


In [68]:
df.groupby('month').agg(total=('sale_amount','sum'))

,total
month,
1,250.50
3,599.99
5,349.99
6,650.00
8,349.99
9,250.50
11,99.99
12,150.00


In [75]:
df.groupby('product_name').agg(total=('sale_amount','sum')).sort_values('total',ascending=False).head(5)

,total
product_name,
Smartwatch,1000.00
Tablet,699.98
Laptop,501.00
Smartphone,300.00
Headphones,199.98


In [78]:
df.groupby("customer_id")["sale_amount"].sum().nlargest(10)

customer_id
101    600.49
105    500.00
109    500.00
108    349.99
104    250.50
103    150.00
107    150.00
102     99.99
106     99.99
Name: sale_amount, dtype: float64

In [79]:
df.groupby("category")["sale_amount"].sum()

category
Electronics    2700.96
Name: sale_amount, dtype: float64